# 라이브러리 호출

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import os

In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

# 데이터 수집

## 크롬 드라이버 생성

In [ ]:
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.maximize_window()
driver.get(url)
time.sleep(3)

## 에브리타임 로그인

In [ ]:
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element_by_name("userid")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element_by_name("password")
et_login.clear()
et_login.send_keys(user_pw)

driver.find_element_by_xpath("""//*[@id="container"]/form/p[3]/input""").click()
time.sleep(3)

In [ ]:
def delete_adv_1():
    while True: # 광고 제거
        try:
            driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
            time.sleep(1.5)
            break
        except:
            continue

In [ ]:
def delete_adv_2():
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
    except:
        pass
    return

In [ ]:
delete_adv_1()

## 연도/학기 시간표 데이터 수집 함수 정의

In [ ]:
def make_course_by_semester(year, semester, start, end):
    text_file = list(pd.read_csv('./course_lst/' + 'course_lst_' + str(year)[2:] + '_' + str(semester) + '.txt', sep = ',').과목명.unique())
    result_df = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
    fault_lst = []
    
    delete_adv_2()
    driver.get(url + '/' + str(year) + '/' + str(semester))
    time.sleep(2.5)
    driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()
    time.sleep(3)
    delete_adv_2()
    time.sleep(1)
    
    for name in tqdm(text_file[start:end]):
        # 1. 테이블 정보 가져오기
        try:
            driver.find_element_by_xpath('//*[@id="subjects"]/div[1]/a[4]').click()
            time.sleep(1)
            print("현재 검색중인 과목 이름 : {}".format(name))
            print()
            course_search = driver.find_element_by_xpath('//*[@id="subjectKeywordFilter"]/div/input')
            course_search.clear()
            time.sleep(0.5)
            course_search.send_keys(name)
        except ElementNotInteractableException or NoSuchElementException or ElementClickInterceptedException:
            fault_lst.append(name)
            continue
        
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="subjectKeywordFilter"]/input').click()
        time.sleep(2.5)
        
        # 2. bs4로 parsing하기
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        contents = []
        tmp = soup.find('div','list').find('tbody').find_all('td')
        
        for i in tmp:
            tmp = i.get_text()
            contents.append(tmp)
        contents = np.array(contents)
        column = []
        tmp = soup.find('div','list').find('thead').find_all('th')
        
        for k in tmp:
            tmp = str(k).lstrip('<th>').split('<div>')
            column.append(tmp[0])
        
        # 3. 데이터프레임 반환
        timetable = pd.DataFrame(contents.reshape(len(contents)//9,9), columns=column)
        
        # 4. 데이터프레임 병합
        result_df = result_df.append(timetable)
        
    # 데이터프레임 중복 제거
    result_df.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
        
    return result_df, fault_lst

In [ ]:
def fault_course(year, semester, lst):
    text_file = lst
    result_df = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
    
    for name in tqdm(text_file):
        print("(실패했던) 현재 검색중인 과목 이름 : {}".format(name))
        print()
        
        # 1. 테이블 정보 가져오기
        driver.get(url + '/' + str(year) + '/' + str(semester))
        time.sleep(3)
        delete_adv_2()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()
        time.sleep(3)
        
        try:
            delete_adv_2()
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="subjects"]/div[1]/a[4]').click()
            time.sleep(3)
            course_search = driver.find_element_by_xpath('//*[@id="subjectKeywordFilter"]/div/input')
            course_search.send_keys(name)
        except ElementNotInteractableException or NoSuchElementException:
            print("또 실패 ?", name)
            continue
        
        time.sleep(1.5)
        driver.find_element_by_xpath('//*[@id="subjectKeywordFilter"]/input').click()
        time.sleep(2)
        
        # 2. bs4로 parsing하기
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        contents = []
        tmp = soup.find('div','list').find('tbody').find_all('td')
        
        for i in tmp:
            tmp = i.get_text()
            contents.append(tmp)
        contents = np.array(contents)
        column = []
        tmp = soup.find('div','list').find('thead').find_all('th')
        
        for k in tmp:
            tmp = str(k).lstrip('<th>').split('<div>')
            column.append(tmp[0])
        
        # 3. 데이터프레임 반환
        timetable = pd.DataFrame(contents.reshape(len(contents)//9,9), columns=column)
        
        # 4. 데이터프레임 병합
        result_df = result_df.append(timetable)
        
    # 데이터프레임 중복 제거
    result_df.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
        
    return result_df

## 데이터 크롤링 시작

### 2016년 1학기 (완료)

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2016_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_16_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_1 = result_2016_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_1 = result_2016_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_1 = result_2016_1.append(result)

# 최종 데이터 프레임 생성
result_2016_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2016_1
result_2016_1.to_csv('2016_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2016년 2학기 (완료)

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2016_2 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_16_2.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 2, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_2 = result_2016_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 2, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_2 = result_2016_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2016, 2, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2016, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2016_2 = result_2016_2.append(result)

# 최종 데이터 프레임 생성
result_2016_2.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2016_2
result_2016_2.to_csv('2016_2.csv', encoding = 'UTF-8-SIG', index = False)

### 2017년 1학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2017_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_17_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_1 = result_2017_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_1 = result_2017_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_1 = result_2017_1.append(result)

# 최종 데이터 프레임 생성
result_2017_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2017_1
result_2017_1.to_csv('2017_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2017년 2학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2017_2 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_17_2.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 2, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_2 = result_2017_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 2, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_2 = result_2017_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2017, 2, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2017, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2017_2 = result_2017_2.append(result)

# 최종 데이터 프레임 생성
result_2017_2.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2017_2
result_2017_2.to_csv('2017_2.csv', encoding = 'UTF-8-SIG', index = False)

### 2018년 1학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2018_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_18_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_1 = result_2018_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_1 = result_2018_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_1 = result_2018_1.append(result)

# 최종 데이터 프레임 생성
result_2018_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2018_1
result_2018_1.to_csv('2018_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2018년 2학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2018_2 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_18_2.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 2, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_2 = result_2018_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 2, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_2 = result_2018_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2018, 2, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2018, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2018_2 = result_2018_2.append(result)

# 최종 데이터 프레임 생성
result_2018_2.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2018_2
result_2018_2.to_csv('2018_2.csv', encoding = 'UTF-8-SIG', index = False)

### 2019년 1학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2019_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_19_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_1 = result_2019_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_1 = result_2019_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_1 = result_2019_1.append(result)

# 최종 데이터 프레임 생성
result_2019_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2019_1
result_2019_1.to_csv('2019_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2019년 2학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2019_2 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_19_2.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 2, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_2 = result_2019_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 2, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_2 = result_2019_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2019, 2, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2019, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2019_2 = result_2019_2.append(result)

# 최종 데이터 프레임 생성
result_2019_2.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2019_2
result_2019_2.to_csv('2019_2.csv', encoding = 'UTF-8-SIG', index = False)

### 2020년 1학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2020_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_20_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_1 = result_2020_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_1 = result_2020_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_1 = result_2020_1.append(result)

# 최종 데이터 프레임 생성
result_2020_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2020_1
result_2020_1.to_csv('2020_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2020년 2학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2020_2 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_20_2.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 2, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_2 = result_2020_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 2, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_2 = result_2020_2.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2020, 2, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2020, 2, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2020_2 = result_2020_2.append(result)

# 최종 데이터 프레임 생성
result_2020_2.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2020_2
result_2020_2.to_csv('2020_1.csv', encoding = 'UTF-8-SIG', index = False)

### 2021년 1학기

In [ ]:
# 빈 데이터프레임 생성 (칼럼)
result_2021_1 = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
input_len = len(list(pd.read_csv('./course_lst/' + 'course_lst_21_1.txt', sep = ',').과목명.unique()))
print(input_len)

# 결과 반환
tmp, fault = make_course_by_semester(2021, 1, 0, 300)
# 실패한 리스트 다시 시도
fault_result = fault_course(2021, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2021_1 = result_2021_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2021, 1, 300, 600)
# 실패한 리스트 다시 시도
fault_result = fault_course(2021, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2021_1 = result_2021_1.append(result)

# 결과 반환
tmp, fault = make_course_by_semester(2021, 1, 600, input_len)
# 실패한 리스트 다시 시도
fault_result = fault_course(2021, 1, fault)
# 결과 병합
result = pd.DataFrame(columns = ['과목번호-분반', '과목명', '교수', '학점', '수업시간/강의실', '학년', '강의평', '담은 인원', '비고'])
result = result.append(tmp)
# 두 데이터 병합
result = result.append(fault_result)
result_2021_1 = result_2021_1.append(result)

# 최종 데이터 프레임 생성
result_2021_1.drop_duplicates(['과목번호-분반', '과목명'], inplace = True)
result_2021_1
result_2021_1.to_csv('2021_1.csv', encoding = 'UTF-8-SIG', index = False)